In [1]:
%load_ext autoreload
%autoreload 2

# Customer Segmentation of "Rate Beer" using Knowledge Graph Embeddings
This notebook uses Pytorch to create a Knowledge Graph Embedding of the customers purchases, usually data scientists try to segment customers in sequences like this using LSTM neural networks.

In [2]:
from clustering.rate_beer_loader import RateBeerLoaderPykeen
from pykeen.metrics.ranking import InverseHarmonicMeanRank
from pykeen.pipeline import pipeline
from pykeen.models import TransE
from pykeen.evaluation.rank_based_evaluator import RankBasedEvaluator

In the `RateBeerLoaderPykeen` object the `ratebeer.txt` file is read and creates a triples

In [3]:
pykeen_rate_beer = RateBeerLoaderPykeen("ratebeer.txt", checkpoint_name="torch_pipeline_checkpoints.pt", limit_reviews_per_reviewer=500)

Found a previous save
Loading Triples from the path downloaded.


In [4]:
training_input = pykeen_rate_beer.get_rate_beer()

Now all the training input data is within `training_input`.
We want a trained network embedding but we also want to be able to have scores as to the value of the graph.
The paper uses a list of

In [5]:
train_transe, test_transe, vali_transe = training_input.split([0.90, 0.05, 0.05], random_state=100)

In [ ]:
for gamma_val in [0.99, 0.9, 0.8]:
    transe_model_pipeline_results = pipeline(training=train_transe,
                                             testing=test_transe, random_seed=100,
                                             validation=vali_transe,
                                             training_loop='sLCWA',
                                             model=TransE,
                                             model_kwargs={"embedding_dim": 50},
                                             training_kwargs=dict(checkpoint_name=f"torch_pipeline_checkpoints_gamma_{gamma_val}.pt",
                                                                  checkpoint_frequency=4, num_epochs=500),
                                             stopper="early",
                                             stopper_kwargs=dict(frequency=1, patience=2, relative_delta=0.001),
                                             lr_scheduler="CosineAnnealingLR", lr_scheduler_kwargs=dict(T_max=10),
                                             evaluator=RankBasedEvaluator,
                                             evaluator_kwargs=dict(metrics=["InverseHarmonicMeanRank"], add_defaults=True))
    transe_model_pipeline_results.save_to_directory(f"transe_trained_gamma_val_{gamma_val}")

INFO:pykeen.pipeline.api:=> no training loop checkpoint file found at '/home/sora4222/.data/pykeen/checkpoints/torch_pipeline_checkpoints_gamma_0.99.pt'. Creating a new file.
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: /home/sora4222/.data/pykeen/checkpoints/best-model-weights-580921d9-10ab-48da-b5c0-2d4c219f9ab0.pt
INFO:pykeen.training.training_loop:=> no checkpoint found at '/home/sora4222/.data/pykeen/checkpoints/torch_pipeline_checkpoints_gamma_0.99.pt'. Creating a new file.


Training epochs on cuda:0:   0%|          | 0/500 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/31667 [00:00<?, ?batch/s]